In [ ]:
import pandas as pd
import json

In [ ]:
with open("results/minimal_bloat__bf0432-007/metrics.json", "r") as file:
    metrics = json.load(file)

In [ ]:
metrics["counters"]

In [ ]:
include_keys = [
    "Total",
    "SubsetDecomposition",
    "Resolution",
    "Unification",
    "SubsumedRemoval1",
    "SubsumedRemoval2",
    "TautologiesRemoval",
]

filtered_durations = [metrics["durations"]["EliminateVar_" + key] for key in include_keys]
elimination = pd.DataFrame(filtered_durations).transpose()
elimination.columns = include_keys

In [ ]:
elimination["MeasurementOverhead"] = elimination["Total"] - elimination[include_keys[1:]].sum(axis=1)

In [ ]:
total = elimination["Total"].copy()
total_sum = total.sum()
ratio = 100_000

def part_of_total(df_col):
    return (df_col.sum() / total_sum) * ratio

def rel_std(df_col):
    return (df_col.std() / df_col.mean()) * 100

aggregation_functions = ["count", "sum", part_of_total, "mean", "median", "std", rel_std, "max", "argmax"]

elimination_summary = elimination.agg(aggregation_functions).astype(int, copy=False)

In [ ]:
elimination_summary.drop("count")

In [ ]:
variable = pd.DataFrame(metrics["durations"]["VarSelection"])
variable.columns = ["VarSelection"]
variable_summary = variable.agg(aggregation_functions).astype(int, copy=False)

In [ ]:
absorbed = pd.DataFrame([metrics["durations"]["RemoveAbsorbedClausesWithConversion"], metrics["series"]["AbsorbedClausesRemoved"]]).transpose()
absorbed.columns = ["AbsorbedRemovalDuration", "AbsorbedClausesRemoved"]
absorbed_summary = absorbed.agg(aggregation_functions).astype(int, copy=False)

In [ ]:
stages_summary = pd.concat([elimination_summary["Total"], variable_summary, absorbed_summary], axis=1) \
                    .rename(columns={"Total": "Elimination"}, index={"part_of_total": "ratio"})
stages_summary.loc["ratio"] //= 100

In [ ]:
stages_summary

In [ ]:
heuristic_data = [
    metrics["series"]["HeuristicScores"],
    metrics["series"]["EliminatedClauses"],
]
heuristic = pd.DataFrame(heuristic_data).transpose()
heuristic_correlation = heuristic[0].corr(heuristic[1])

In [ ]:
duration_ratio = 1000
summary_data = {
    "TotalVars": metrics["counters"]["TotalVars"],
    "EliminatedVars": metrics["counters"]["EliminatedVars"],
    "TotalClauses": metrics["counters"]["TotalClauses"],
    "RemovedClauses": metrics["counters"]["RemovedClauses"],
    "RemovedAbsorbedClauses": metrics["counters"]["AbsorbedClausesRemovedTotal"],
    "HeuristicCorrelation": heuristic_correlation,
    "TotalDuration": metrics["durations"]["EliminateVars"],
    "VarSelection": stages_summary["VarSelection"].loc["sum"],
    "Elimination": stages_summary["Elimination"].loc["sum"],
    "AbsorbedRemoval": stages_summary["AbsorbedRemovalDuration"].loc["sum"],
}
summary = pd.DataFrame(summary_data)
summary[["TotalDuration", "VarSelection", "Elimination", "AbsorbedRemoval"]] //= duration_ratio

In [ ]:
summary